In [2]:
# Step 1
import pandas as pd
from pathlib import Path
import numpy as np

In [3]:
# Files needed
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

In [4]:
# Read store into Pandas DataFrames
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

In [5]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df = pd.DataFrame(school_data_complete)


In [6]:
# calculate the number of unique schools - using (nunique) in this case, on the complete df
unique_schools = school_data_complete_df["school_name"].nunique()

In [7]:
# Idea to use numpy sum  - From dan12345, https://stackoverflow.com/questions/20995196/pandas-counting-and-summing-specific-conditions
# lga is local government area, so the entire dataset

lga_reading = np.sum(school_data_complete_df["reading_score"] >=50)/school_data_complete_df["reading_score"].count()*100

In [8]:
lga_maths = np.sum(school_data_complete_df["maths_score"] >=50)/school_data_complete_df["maths_score"].count()*100

In [9]:
lga_overall = school_data_complete_df.query("maths_score >=50 and reading_score >=50")["reading_score"].count()/school_data_complete_df["reading_score"].count()*100

In [10]:
#Calculate the total number of students
total_students = school_data_complete_df["Student ID"].nunique()

In [11]:
# Total school budgets for LGA area. Drop duplicates info from: # https://www.w3schools.com/python/pandas/ref_df_drop_duplicates.asp#:~:text=The%20drop_duplicates()%20method%20removes,considered%20when%20looking%20for%20duplicates.
total_budget_2 = school_data_complete_df.drop_duplicates("budget")
all_schools_budget = total_budget_2["budget"].sum()


In [12]:
# Ave reading score
all_students_reading = school_data_complete_df["reading_score"].mean()

In [13]:
# Ave maths score
all_students_maths = school_data_complete_df["maths_score"].mean()

In [14]:
#Create a dataframe to hold the above results

data = [{"Total Schools":unique_schools, "Total Students":total_students, "Total Budget":all_schools_budget, "Average Maths Score":all_students_maths, "Average Reading Score":all_students_reading, "% Passing Maths":lga_maths, "% Passing Reading":lga_reading, "% Overall Passing":lga_overall}]
lga_summary = pd.DataFrame(data)
lga_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


In [15]:
#Optional: give the displayed data cleaner formatting

lga_summary["Total Budget"] = lga_summary["Total Budget"].map("${:,.2f}".format)

lga_summary


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [16]:
#Schools summary

In [17]:
#School reading and maths averages

average_df = school_data_complete_df[["school_name", "reading_score", "maths_score"]]
average_df.groupby(["school_name"]).mean()
school_data_means_df = pd.DataFrame(average_df.groupby(["school_name"]).mean())




In [18]:
school_data_by_school = pd.merge(school_data_means_df, school_data, how="left", on=["school_name", "school_name"])
school_data_by_school = school_data_by_school[['school_name','type','size','budget', 'maths_score' , 'reading_score']] 
school_data_by_school.rename(columns={'school_name':"School Name","type":"Type", 'size':"Total Students", "budget":"Budget",'maths_score':"Average Maths Score", 'reading_score':'Average Reading Score'}, inplace = True)
school_data_by_school
#school_data_by_school.set_index("School_Name", inplace=True)
PerStudent = school_data_by_school['Budget']/school_data_by_school['Total Students']
school_data_by_school.insert(4,"Per Student Budget",PerStudent,True)

In [19]:
passread = []
passmaths = []
passboth = []
schoollist = school_data_by_school['School Name'].tolist()
for x in schoollist:
    newdf = school_data_complete_df[school_data_complete_df['school_name'] == x]
    passread.append(np.sum(newdf["reading_score"] >=50)/newdf["reading_score"].count()*100)
    passmaths.append(np.sum(newdf["maths_score"] >=50)/newdf["maths_score"].count()*100)
    passboth.append(newdf.query("maths_score >=50 and reading_score >=50")["reading_score"].count()/newdf["reading_score"].count()*100)

school_data_by_school.insert(7,"% Passing Reading",passread,True)
school_data_by_school.insert(7,"% Passing Maths",passmaths,True)
school_data_by_school.insert(9,"% Overall Passing",passboth,True)

school_data_by_school


,School Name,Type,Total Students,Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
5,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
6,Holden High School,Independent,427,248087,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
7,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
9,Pena High School,Independent,962,585858,609.0,72.088358,71.613306,91.683992,86.590437,79.209979


In [20]:
#Top Performing Schools (By % Overall Passing)
#Sort and display the top five performing schools by % overall passing.
school_data_by_school.sort_values('% Overall Passing', ascending=False).head()


,School Name,Type,Total Students,Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
4,Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
1,Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
0,Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
14,Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
10,Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


In [21]:
#Bottom Performing Schools (By % Overall Passing)
#Sort and display the five worst-performing schools by % overall passing.
school_data_by_school.sort_values('% Overall Passing', ascending=True).head()

,School Name,Type,Total Students,Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
5,Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
7,Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
13,Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
3,Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


In [22]:
#Table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.
#Pandas series for each year.
only_nine = student_data.loc[student_data["year"] == 9 ]
only_ten = student_data.loc[student_data["year"] == 10]
only_eleven = student_data.loc[student_data["year"] == 11]
only_twelve = student_data.loc[student_data["year"] == 12]

#Group each series by school
nine_m_ave = only_nine.groupby(['school_name']).mean()["maths_score"]
ten_m_ave = only_ten.groupby(['school_name']).mean()["maths_score"]
eleven_m_ave = only_eleven.groupby(['school_name']).mean()["maths_score"]
twelve_m_ave = only_twelve.groupby(['school_name']).mean()["maths_score"]

#Combine the series into a dataframe
Maths_scores_by_level = pd.DataFrame([nine_m_ave, ten_m_ave, eleven_m_ave, twelve_m_ave])
Maths_scores_by_level = Maths_scores_by_level.T
Maths_scores_by_level.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']
                                      
Maths_scores_by_level


C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\2257228825.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nine_m_ave = only_nine.groupby(['school_name']).mean()["maths_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\2257228825.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ten_m_ave = only_ten.groupby(['school_name']).mean()["maths_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\2257228825.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or sele

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


In [23]:
# Reading Score by Year - Same operations as above for reading scores

nine_r_ave = only_nine.groupby(['school_name']).mean()["reading_score"]
ten_r_ave = only_ten.groupby(['school_name']).mean()["reading_score"]
eleven_r_ave = only_eleven.groupby(['school_name']).mean()["reading_score"]
twelve_r_ave = only_twelve.groupby(['school_name']).mean()["reading_score"]


Read_scores_by_level = pd.DataFrame([nine_r_ave, ten_r_ave, eleven_r_ave, twelve_r_ave])
Read_scores_by_level = Read_scores_by_level.T
Read_scores_by_level.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']
                                      
Read_scores_by_level


C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\415275237.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nine_r_ave = only_nine.groupby(['school_name']).mean()["reading_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\415275237.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ten_r_ave = only_ten.groupby(['school_name']).mean()["reading_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_13380\415275237.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or selec

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


In [24]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student)
# Use code and bins provided

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_spending_df = school_data_by_school

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=labels, include_lowest=True)

#pd.cut to categorise spending based on the bins.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

spending_summary = pd.DataFrame([spending_math_scores, spending_reading_scores, spending_passing_math, spending_passing_reading, overall_passing_spending]).round(2)

spending_summary = spending_summary.T

spending_summary



,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


In [25]:
# Scores by School Size
# Perform the same operations as above, based on school size.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

school_size_df = school_data_by_school

school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], size_bins, labels=size_labels, include_lowest=True)

#pd.cut to categorise spending based on the bins.
size_math_scores = school_spending_df.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = school_spending_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_spending_df.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = school_spending_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_size = school_spending_df.groupby(["School Size"])["% Overall Passing"].mean()

size_summary = pd.DataFrame([size_math_scores, size_reading_scores, size_passing_math, size_passing_reading, overall_passing_size])

size_summary = size_summary.T

size_summary


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


In [26]:
# Scores by School Type
# Perform the same operations as above, based on school type 

#type_bins = ["Government","Independent"]
#type_labels = ["Government","Independent"]

#pd.cut to categorise spending based on the bins.
type_math_scores = school_data_by_school.groupby(["Type"])["Average Maths Score"].mean()
type_reading_scores = school_data_by_school.groupby(["Type"])["Average Reading Score"].mean()
type_passing_math = school_data_by_school.groupby(["Type"])["% Passing Maths"].mean()
type_passing_reading = school_data_by_school.groupby(["Type"])["% Passing Reading"].mean()
overall_passing_type = school_data_by_school.groupby(["Type"])["% Overall Passing"].mean()

type_summary = pd.DataFrame([type_math_scores, type_reading_scores, type_passing_math, type_passing_reading, overall_passing_type])

type_summary = type_summary.T

type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340


Written Report

The data which has been analysed represents area-wide standardised test result information on maths and reading scores for the schools in the Local Government Area (LGA). 

The data has not been analysed stastically - so all observations are based on differences between observations, with no reference to standard deviations or stastical significance. 

Overall, Independent school students perform slightly bettter then Government school students.

Students from smaller schools perform better than students from larger schools.

Counterintutively, students from schools which receive a smaller amount of funding per student perform better than schools which receive a larger dollar value per student. This result might be being confounded by the size of the school - further analysis is required.